In [2]:
# !pip install sqlite3
# !pip install time
# !pip install random
# !pip install psutil
# !pip install os

só tem que conferir se alguns deles não são nativos do python

In [3]:
import sqlite3
import time
import random
import psutil
import os